<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/PatternMining/Process_for_structure_pm(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mise en forme des donnees pour le *Pattern Mining*

### Imports

In [0]:
import pandas as pd  
import numpy as np
import warnings

### Recuperation et preparation des donnnees

In [2]:
#Chargement du jeu de donnees
data_child = pd.read_csv('data_madeleine_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col = False)

# Supression de la colonne qui duplique l'index
data_child = data_child.drop(columns = 'Unnamed: 0') 

# Recuperation de la partie phonetique des intervention de l'enfant
data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])

# Conservation des deux colonnes avec des informations pertinantes 
data_child = data_child[['age','contenu']]

# Creer un indexage a partir de cet ordre
data_child['index']=[ i for i in range(len(data_child))]
data_child= data_child.set_index('index')

# Creation d'un numpy utile par la suite pour les boucles
ages = data_child.age.unique()

# Indexage par l'age de l'enfant
age = data_child['age']

l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))

index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
data_child.set_index(index,inplace=True)

# Supression de la colonne age car present en index
data_child=data_child[['contenu']]

data_child.head()

contenu
age     index        
1_00_05 0          eː
        1          bɛ
        2        vœga
        3       gatœ̃
        4        kaka

### Demarches pour la division de chaque enregistrement en 4 parties

In [0]:
# Sépare en mots au lieu de phrases
mots = {}
for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    
# se parcourt avec un age en parametre
#mots["1_00_05"]

In [0]:
# Sépare en 4 "enfants" chaque enregistrement
enregistrement = {}
sizes = []
for i in list(ages):
    # recupere tout les mots a la i eme date
    #data = data_child['contenu'].loc[i].values
    data = mots[i]
    # separe les mots en 4 groupes ( 3 goupes de meme dim et le dernier avec le reste)
    if len(data)<4:
        warnings.warn("\nAttention un l'enregistrement "+ i + " contient moins de 4 mots")
    s = len(data)//4
    sizes.append(s)
    enfant1 = data[:s]
    enfant2 = data[s:2*s]
    enfant3 = data[2*s:3*s]
    enfant4 = data[3*s:]
    # creer une variable texte de chaque goupe
    #enfant1 = ' '.join(enfant1)
    #enfant2 = ' '.join(enfant2)
    #enfant3 = ' '.join(enfant3)
    #enfant4 = ' '.join(enfant4)
    
    enfant1 = tuple(enfant1)
    enfant2 = tuple(enfant2)
    enfant3 = tuple(enfant3)
    enfant4 = tuple(enfant4)
    
    # sépare en 4 enfants
    enregistrement[i] = [enfant1, enfant2, enfant3, enfant4]
    
# se parcourt avec un age en parametre
#enregistrement["1_00_05"]

In [5]:
# Pour avoir une idée du nombre de mots par enregistrement après répartition
pd.DataFrame(sizes[:-1]).describe().T

,count,mean,std,min,25%,50%,75%,max
0,18.0,322.444444,223.518711,68.0,126.75,261.0,449.75,737.0


In [6]:
# Crée une DataFrame de toutes les donnees
tab_4groupes = []
#sizes = []
for i in list(ages):
    t = enregistrement[i]
    #s = len(t)
    tab_4groupes.append(t)
    #sizes.append(s)
    
tab_madeleine = pd.DataFrame(tab_4groupes)

etiquettes = ["madeleine_enfant1",
              "madeleine_enfant2", 
              "madeleine_enfant3", 
              "madeleine_enfant4"]

tab_madeleine.columns = etiquettes

tab_madeleine.head()
# tab_madeleine.loc[1] recherche par numero de l'enregistrement
# tab_madeleine[1] recherche par numero de l'enfant

,madeleine_enfant1,madeleine_enfant2,madeleine_enfant3,madeleine_enfant4
0,"(eː, bɛ, vœga, gatœ̃, kaka, dœjøkɛka, m, taka,...","(gla, ga, ga, gøga, gøga, ga, gaga, a, deda, œ...","(gaga, gaga, gagaga, ga, mœmœmœmœ, ala, adla, ...","(œmøː, œmøː, gaga, gagaga, daɥy, aga, œm, œm, ..."
1,"(ja, aljɛla, jaka, meja, agø, ga, beːja, ʃa, o...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, aja, ja, ...","(ʃa, ejaː, edjadjajaaa, edahahaoga, u, ajne, e...","(ada, aja, gɛʒeʒa, ajəda, dedaː, da, ega, ja, ..."
2,"(je, ʃidɔm, øʃi, alapa, ʃː, ʃ, a, ʃa, øː, aaː,...","(bɛ, y, øː, ʃ, mememebuja, eaa, iɲa, ia, ʃɛ̃, ...","(ʃa, ʒy, aːaːɛ, ɛɛ, sə, wauːua, a, χ, wa, waa,...","(bidun, bida, bjɛtna, bebøː, dakula, daguba, m..."
3,"(evøpuʃa, gegedɔ̃, dɔ, ty, hɛsys, ejə, ʃa, gøt...","(mama, mama, bekaliʃaʃa, øtatɛ, mama, løʃa, pe...","(ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃa, ʃːa, səba, ʃa, ʃa...","(lelɛ, lala, lala, uu, iː, jejejejejee, eː, mː..."
4,"(i, lɛ, vwa, a, aː, ti, vwa, aː, ty, pø, t, as...","(wikɔ, ejdøwa, ɛʃa, pøpa, ɛøgødididitetɛe, bøb...","(wœ, wa, wawa, pwapwa, awœ, œœ, ba, m, jeje, e...","(jije, ini, œː, i, ʃi, uba, veve, tilele, nine..."


### Addaptation de la structure

In [0]:
# transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
# doit etre de cette forme:
# madeleine_enfant1 = transactions
# un enregistrement = ('a', 'b', 'c')
# un mot = 'a'

In [0]:
madeleine_enfant1 = tuple(tab_madeleine["madeleine_enfant1"][:-1])
madeleine_enfant2 = tuple(tab_madeleine["madeleine_enfant2"][:-1])
madeleine_enfant3 = tuple(tab_madeleine["madeleine_enfant3"][:-1])
madeleine_enfant4 = tuple(tab_madeleine["madeleine_enfant4"][:])

In [9]:
print(type(madeleine_enfant1))
#print(madeleine_enfant1)

<class 'tuple'>


### PATTERN MINING

In [10]:
!pip install pymining
from pymining import itemmining
from pymining import assocrules
from pymining import seqmining

In [11]:
min_supp = 10
relim_input = itemmining.get_relim_input(madeleine_enfant1)
item_sets = itemmining.relim(relim_input, min_support=min_supp) # min_support= nbr au pif < lignes
item_sets

{frozenset({'dɑ̃'}): 10,
 frozenset({'dɑ̃', 'l'}): 10,
 frozenset({'dɑ̃', 'l', 'le'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa', 'se', 'ɛ'}): 10,
 frozenset({'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se', 'ɛ'}): 10,
 frozenset({'a', 'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se', 'ɛ'}): 10,
 frozenset({'a', 'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa', 'se', 'ɛ'}): 10,
 frozenset({'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}): 10,
 frozenset({'a', 'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}): 10,
 frozenset({'a', 'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}): 10,
 frozenset({'dɑ̃', 'l', 'le', 'lə', 'mamɑ̃', 'sa', 'ɛ'}): 10,
 frozenset({'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'ɛ'}): 10,
 frozenset({'a', 'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa',

In [12]:
rules = assocrules.mine_assoc_rules(item_sets, min_support=min_supp, min_confidence=0.5)
rules

[(frozenset({'a', 'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'se', 'ɛ'}),
  frozenset({'sa'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'se', 'ɛ'}),
  frozenset({'a', 'sa'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'l', 'la', 'le', 'mamɑ̃', 'se', 'ɛ'}),
  frozenset({'a', 'lə', 'sa'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'l', 'le', 'mamɑ̃', 'se', 'ɛ'}),
  frozenset({'a', 'la', 'lə', 'sa'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'le', 'mamɑ̃', 'se', 'ɛ'}),
  frozenset({'a', 'l', 'la', 'lə', 'sa'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'le', 'mamɑ̃', 'ɛ'}),
  frozenset({'a', 'l', 'la', 'lə', 'sa', 'se'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'le', 'ɛ'}),
  frozenset({'a', 'l', 'la', 'lə', 'mamɑ̃', 'sa', 'se'}),
  10,
  1.0),
 (frozenset({'dɑ̃', 'ɛ'}),
  frozenset({'a', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}),
  10,
  1.0),
 (frozenset({'ɛ'}),
  frozenset({'a', 'dɑ̃', 'l', 'la', 'le', 'lə', 'mamɑ̃', 'sa', 'se'}),
  10,
  0.7142857142857143),
 (frozenset({'dɑ̃'}),
  frozenset({'a', 

In [13]:
#seqs = ( 'caabc', 'abcb', 'cabc', 'abbca')
seqs = madeleine_enfant1

freq_seqs = seqmining.freq_seq_enum(seqs, 9)
for seq in freq_seqs:
    print (seq)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
(('e', 'a', 'l', 'sa'), 10)
(('e', 'a', 'e', 'e', 'l', 'le', 'a', 'sa'), 9)
(('sa', 'la', 'lə', 'la', 'l'), 10)
(('lə', 'la', 'la', 'e', 'le', 'e'), 9)
(('a', 'e', 'e', 'i', 'm'), 9)
(('e', 'e', 'e', 'sa', 'e', 'se', 'a', 'a'), 9)
(('e', 'dɑ̃', 'lə', 'a', 'lə', 'a', 'lə'), 9)
(('l', 'lə', 'e', 'le', 'e'), 9)
(('a', 'a', 'a', 'e', 'e', 'ɛ'), 9)
(('ɛ', 'le', 'a', 'a'), 9)
(('a', 'a', 'a', 'a', 'e', 'm', 'a', 'a', 'a'), 9)
(('a', 'se', 'l', 'l', 'lə'), 9)
(('e', 'a', 'i', 'la', 'e', 'e', 'la'), 9)
(('a', 'la', 'la', 'a', 'e', 'a', 'lə'), 9)
(('a', 'a', 'sa', 'ɛ', 'l', 'la'), 9)
(('pa', 'e', 'la', 'e', 'e', 'lə'), 9)
(('sa', 'e', 'l', 'le', 'la', 'a', 'a'), 9)
(('lə', 'la', 'e', 'a', 'sa'), 9)
(('e', 'sa', 'l', 'a', 'l', 'e'), 9)
(('e', 'e', 'e', 'a', 'l', 'lə', 'a', 'l'), 9)
(('lə', 'mamɑ̃', 'lə', 'lə', 'lə', 'sa'), 9)
(('a', 'e', 'e', 'la', 'a', 'l', 'l'), 9)
(('lə', 'a', 'la', 'la', 'a', 'a', 'e', 'e'), 9)
(('